#### 전이학습
- 거대 이미지 데이터셋으로 사전학습된 모델을 활용
- ImageNet 데이터베이스 기반 모델들 사용
- 사용법
    - 모델의 특징추출부분(CNN) 재사용
    - 모델의 학습, 분류 부분(FC)은 커스텀화

[1] 모듈 로딩<hr>

In [4]:
import torch
import torch.nn as nn
import torchvision.models as models
from torchinfo import summary

[2] 사전학습된 모델 로딩 및 설정<hr>
- 모델 사용 가중치 및 바이어스 설정
- 층별 가중치 및 바이어스의 비활성화 설정

In [5]:
# 사전학습된 모델 로딩
# 가중치 안주면 랜덤으로 부여됨
model= models.vgg16(weights=models.VGG16_Weights.IMAGENET1K_V1)

In [6]:
model.parameters

<bound method Module.parameters of VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size

In [8]:
# 역전파 시에 업데이트 되지 않도록 설정
for n, p in model.named_parameters():
    p.resquires_grad=False

In [10]:
# 첫번째 sequnetial 층
model.features

Sequential(
  (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (1): ReLU(inplace=True)
  (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (3): ReLU(inplace=True)
  (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (6): ReLU(inplace=True)
  (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (8): ReLU(inplace=True)
  (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (11): ReLU(inplace=True)
  (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (13): ReLU(inplace=True)
  (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (15): ReLU(inplace=True)
  (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (17): Conv2d(256, 512, kernel_si

In [13]:
class MYCNN(nn.Module):
    def __init__(self) -> None:
        super().__init__()
        self.features=nn.Sequential(
            nn.Conv2d(),
            nn.Conv2d(),
            nn.MaxPool2d(),
            nn.Conv2d(),
            nn.Conv2d(),
            nn.MaxPool2d(),
            nn.Conv2d(),
            nn.Conv2d(),
            nn.MaxPool2d()
        )

        self.classifier=nn.Sequential(
            nn.Linear(3,5),
            nn.ReLU(),
            nn.Linear(3,5),
            nn.ReLU()
        )
    def forward(self, data):
        out=self.features(data)
        out=self.classifier(out.view(out.shape[0],-1))
        return out

In [14]:
# model.classifier=nn.Linear(in_features=25088, out_features=1000, bias=True)
# 이런 식으로 직접 설정도 가능

Sequential(
  (0): Linear(in_features=25088, out_features=4096, bias=True)
  (1): ReLU(inplace=True)
  (2): Dropout(p=0.5, inplace=False)
  (3): Linear(in_features=4096, out_features=4096, bias=True)
  (4): ReLU(inplace=True)
  (5): Dropout(p=0.5, inplace=False)
  (6): Linear(in_features=4096, out_features=1000, bias=True)
)